In [ ]:
import torch

In [ ]:
# OS GPU driver + CUDA
!nvidia-smi

# torch cuda bindings
print(f"torch cuda availability: {torch.cuda.is_available()}")

In [ ]:
torch.cuda.get_arch_list()

In [ ]:
mu = '\u03BC'
sigma = '\u03C3'

# 1. Model Architecture

![title](./generator.jpg)


![title](./discriminator.jpg)

#### Residual Block



In [ ]:
import torch
from torch import nn


class ResidualBlock(nn.Module):
    def __init__(self, in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1):
        super(ResidualBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.conv_1 = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=(self.kernel_size, self.kernel_size),
                               stride=(self.stride, self.stride), padding=(self.padding, self.padding))
        self.conv_2 = nn.Conv2d(self.out_channels, self.out_channels, kernel_size=(self.kernel_size, self.kernel_size),
                               stride=(self.stride, self.stride), padding=(self.padding, self.padding))
        self.bn_1 = nn.BatchNorm2d(self.out_channels)
        self.bn_2 = nn.BatchNorm2d(self.out_channels)
        self.act = nn.PReLU()

    def forward(self, x):
        # first
        t = self.conv_1(x)
        t = self.bn_1(t)
        t = self.act(t)

        # second
        t = self.conv_2(t)
        t = self.bn_2(t)

        return torch.add(x, t)


# Test
tensor = torch.randn(1, 64, 224, 224).to('cuda')
model = ResidualBlock(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1).to('cuda')
out = model(tensor)
print(f"output tensor shape {out.shape}")
print(f"input tensor stats: {mu}: {tensor.mean():.2f}, {sigma}: {tensor.std():.2f}")
print(f"output tensor stats: {mu}: {out.mean():.2f}, {sigma}: {out.std():.2f}")


#### Upscale Block

In [ ]:
class UpscaleBlock(nn.Module):
    def __init__(self, in_channels=64, out_channels=256, scale_factor=2, kernel_size=3, stride=1, padding=1):
        super(UpscaleBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.scale = scale_factor
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.conv_1 = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=(self.kernel_size, self.kernel_size),
                               stride=(self.stride, self.stride), padding=(self.padding, self.padding))
        self.act = nn.PReLU()
        self.shuffler = nn.PixelShuffle(scale_factor)

    def forward(self, x):
        x = self.conv_1(x)
        x = self.act(x)
        x = self.shuffler(x)

        return x


# Test
tensor = torch.randn(1, 64, 224, 224).to('cuda')
block_1 = UpscaleBlock(in_channels=64, out_channels=64 * 2 ** 2, scale_factor=2, kernel_size=3, stride=1, padding=1).to('cuda')
block_2 = UpscaleBlock(in_channels=64, out_channels=64 * 2 ** 2, scale_factor=2, kernel_size=3, stride=1, padding=1).to('cuda')
out_1 = block_1(tensor)
out_2 = block_2(out_1)
print(f"output tensor shape {out_2.shape}")
print(f"input tensor stats: {mu}: {tensor.mean():.2f}, {sigma}: {tensor.std():.2f}")
print(f"output tensor stats: {mu}: {out_2.mean():.2f}, {sigma}: {out_2.std():.2f}")


#### Generator Network

In [ ]:
import numpy as np

class SRResNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_of_res_blocks=5, scale_factor=4, step_scale_factor=2,
                 stride=1, padding=1):
        super(SRResNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_of_res_blocks = num_of_res_blocks
        self.scale_factor = scale_factor
        self.step_scale_factor = step_scale_factor
        if not np.emath.logn(step_scale_factor, scale_factor).is_integer():
            raise ValueError(f"scale_factor {scale_factor} cannot be reached with stacked layers, each step-scale by {step_scale_factor}")
        
        # we need to translate the scale factor into the number of scaling layers
        self.num_of_scaling_layers = (scale_factor // step_scale_factor) - 1
        self.stride = stride
        self.padding = padding

        # low frequency information extraction
        self.in_block = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=9, stride=stride, padding=4),
            nn.PReLU()
        )
        
        # high frequency information extraction
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(in_channels=64, out_channels=64, padding=padding) for _ in range(num_of_res_blocks)]
        )
        
        # high frequency information fusion
        self.after_res_block = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=stride, padding=padding),
            nn.BatchNorm2d(64)
        )
        
        # zooming 
        self.upscale_layers = nn.Sequential(
            UpscaleBlock(in_channels=64, out_channels=64 * step_scale_factor * step_scale_factor, scale_factor=step_scale_factor, kernel_size=3, stride=stride,
                         padding=padding),
            *[UpscaleBlock(in_channels=64, out_channels=64 * step_scale_factor * step_scale_factor, scale_factor=step_scale_factor, kernel_size=3, stride=stride,
                           padding=padding) for _ in range(self.num_of_scaling_layers)]
        )
        
        # reconstruction
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=9, stride=stride, padding=4)
        
    def forward(self, x):
        x = self.in_block(x)
        z = self.residual_blocks(x)
        z = self.after_res_block(z)
        z = torch.add(z, x)
        z = self.upscale_layers(z)
        return self.final_conv(z)


# Test
tensor = torch.randn(1, 3, 224, 224).to('cuda')
model = SRResNet(in_channels=3, out_channels=3, scale_factor=4, stride=1, padding=1).to('cuda')
out = model(tensor)
print(f"output tensor shape {out.shape}")
print(f"input tensor stats: \u03BC: {tensor.mean():.2f}, \u03C3: {tensor.std():.2f}")
print(f"output tensor stats: \u03BC: {out.mean():.2f}, \u03C3: {out.std():.2f}")



#### Discriminator ConvBlock

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1):
        super(ConvBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        self.conv = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=self.kernel_size,
                               stride=self.stride, padding=self.padding)
        self.bn = nn.BatchNorm2d(self.out_channels)
        self.act = nn.LeakyReLU(0.2, True)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.act(x)
    
# Test
tensor = torch.randn(1, 3, 224, 224).to('cuda')
model = ConvBlock(in_channels=3, out_channels=64, stride=2, padding=1).to('cuda')
out = model(tensor)
print(f"output tensor shape {out.shape}")
print(f"input tensor stats: \u03BC: {tensor.mean():.2f}, \u03C3: {tensor.std():.2f}")
print(f"output tensor stats: \u03BC: {out.mean():.2f}, \u03C3: {out.std():.2f}")      

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(Discriminator, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # high frequency features
        self.in_block = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU()
        )
        
        # conv blocks
        self.conv_blocks = nn.Sequential(
            ConvBlock(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=1),
            ConvBlock(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels=128, out_channels=128, kernel_size=3, stride=2, padding=1),
            ConvBlock(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=1),
            ConvBlock(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels=512, out_channels=512, kernel_size=3, stride=2, padding=1)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 6 * 6, 1024),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.in_block(x)
        x = self.conv_blocks(x)
        x = torch.flatten(x, start_dim=1)
        return self.classifier(x)
        

# Test
tensor = torch.randn(1, 3, 96, 96).to('cuda')
model = Discriminator(in_channels=3, out_channels=1).to('cuda')
out = model(tensor)
print(f"output tensor shape {out.shape}")
print(f"input tensor stats: \u03BC: {tensor.mean():.2f}, \u03C3: {tensor.std():.2f}")
print(f"output tensor stats: \u03BC: {out.mean():.2f}, \u03C3: {out.std():.2f}")      

# 2. Loss Functions

In [ ]:
class PerceptualLoss(nn.Module):
    